In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 31.0MB/s 
     |████████████████████████████████| 3.3MB 38.0MB/s 
     |████████████████████████████████| 901kB 32.8MB/s 
     |████████████████████████████████| 1.2MB 27.9MB/s 


In [ ]:
import torch
#torch.cuda.empty_cache()
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences

from transformers import *
import numpy as np
import json

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
print(device)
# print(torch.cuda.get_device_name(0))

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_folder = "/content/drive/My Drive/CS182-Spring2020-NLP-Project/"

Mounted at /content/drive


In [ ]:
data = []
with open(root_folder + 'dataset/test_data.jsonl', 'r') as file:
    data = [json.loads(jline) for jline in file.read().splitlines()]

In [ ]:
labels = [int(review["stars"])-1 for review in data]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False, remove_space=False)
data = [tokenizer.tokenize("[CLS] " + review["text"] + " [SEP]") for review in data]
print ("Tokenize the first sentence:")
print (data[0])


Tokenize the first sentence:
['[CLS]', 'i', 'great', 'experience', 'ik', '##on', 'i', 'went', 'washing', 'cutting', 'bloody', '##ing', 'curling', 'hair', 'friends', 'birthday', 'that', 'i', 'paid', 'total', '70', 'tax', 'they', 'specific', 'flat', 'rate', 'hair', '##dre', '##sser', 'salon', 'hair', '##dre', '##sser', 'whose', 'name', 'i', 'believe', 'michael', 'really', 'best', 'make', 'sure', 'everything', 'looked', 'i', 'wanted', 'it', 'too', 'bad', 'phone', 'died', 'i', 'left', 'i', 'take', 'picture', 'amazing', 'hair', 'looked', 'i', 'remember', 'getting', 'many', 'compliment', '##s', 'hair', 'lo', '##oon', '##g', 'time', 'i', 'definitely', 'go', 'back', 'next', 'cut', 'whenever', 'i', 'need', 'get', 'hair', 'done', 'nicely', 'really', 'worth', 'money', 'i', 'also', 'liked', 'friendly', 'service', '[SEP]']


In [ ]:
MAX_LEN = 512

In [ ]:
data = [tokenizer.convert_tokens_to_ids(x) for x in data]

In [ ]:
data = pad_sequences(data, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
attention_masks = []

for seq in data:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [ ]:
data = torch.tensor(data)
labels = torch.tensor(labels)
attention_masks = torch.tensor(attention_masks)

In [ ]:
batch_size = 1

train_data = TensorDataset(data, attention_masks, labels)
train_sampler = RandomSampler(train_data)
prediction_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
model = BertForSequenceClassification.from_pretrained("/content/drive/My Drive/BERT_lr_26", num_labels=5)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
model.eval()

total_diff = 0
correct = 0
count = 0

predictions , true_labels = [], []

for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  
  with torch.no_grad():
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  predictions.append(logits)
  true_labels.append(label_ids)

  for i in range(len(logits)):
    pred = np.argmax(logits[i])
    total_diff += abs(pred - int(label_ids[i]))
    correct += 1 if pred == int(label_ids[i]) else 0
    count += 1
  print("Predicted:", pred + 1,
      "\tActual:", int(label_ids[i]) + 1,
      "\tDiff:", total_diff,
      "\tAccuracy:", correct / count)

Streaming output truncated to the last 5000 lines.
Predicted: 1 	Actual: 1 	Diff: 2051 	Accuracy: 0.8073490205169815
Predicted: 1 	Actual: 1 	Diff: 2051 	Accuracy: 0.8073713490959666
Predicted: 4 	Actual: 3 	Diff: 2052 	Accuracy: 0.8072777842160158
Predicted: 5 	Actual: 4 	Diff: 2053 	Accuracy: 0.8071842410196988
Predicted: 5 	Actual: 5 	Diff: 2053 	Accuracy: 0.8072065809292087
Predicted: 3 	Actual: 2 	Diff: 2054 	Accuracy: 0.8071130676552364
Predicted: 1 	Actual: 1 	Diff: 2054 	Accuracy: 0.8071354106336152
Predicted: 1 	Actual: 1 	Diff: 2054 	Accuracy: 0.8071577484364142
Predicted: 3 	Actual: 3 	Diff: 2054 	Accuracy: 0.8071800810654314
Predicted: 5 	Actual: 5 	Diff: 2054 	Accuracy: 0.807202408522464
Predicted: 1 	Actual: 1 	Diff: 2054 	Accuracy: 0.8072247308093088
Predicted: 5 	Actual: 5 	Diff: 2054 	Accuracy: 0.807247047927761
Predicted: 1 	Actual: 1 	Diff: 2054 	Accuracy: 0.8072693598796157
Predicted: 5 	Actual: 5 	Diff: 2054 	Accuracy: 0.8072916666666666
Predicted: 3 	Actual: 3 	Di